In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

<h1><center> “Good Dinner AI”</center></h1>
<h3>Use case</h3>

“As a user of Good Dinner, I can:
* post reviews in the form of comments.
* post photos taken in the restaurant.”

“As Good Dinner, I wish to:
* Detect the topics of dissatisfaction in the comments posted on the platform.
* Automatically label photos posted on the platform. For example, identifying photos related to food or decor inside or outside the restaurant.” 

<h3>Scope of the project</h3>
Preliminary study of feature “Detect the topics of dissatisfaction” and “Automatically label posted photos”

<h3>Dataset and Data Collection</h3>

* Problem: Insufficient data available on the Good Dinner platform.
* Solution: Use an existing dataset.
>> * Link to the dataset: https://www.yelp.com/dataset (Contains general information (e.g., type of cuisine) and consumer reviews of different restaurants) 
>> * To fetch new data use the yelp API

<h3> Methodology and process</h3>

* analyze the comments in order to detect topics of dissatisfaction:
>> * pre-process text data
>> * use dimension reduction techniques
>> * visualize high-dimensional data
* analyze photos in order to determine their categories 
>> * pre-process images
>> * use dimension reduction techniques
>> * visualize high-dimensional data
* collect sample collection (approximately 200 restaurants) of data via the Yelp API:
>> * retrieve only the necessary fields
>> * store the results in a usable file format (e.g., csv)
<h3>Tools</h3>

* Python and specialized NLP/CV libraries
* Jupyter Notebook and Voilà package

<h2><center>LOAD DATA </center></h2>

Yelp Fusion API

https://docs.developer.yelp.com/docs/fusion-intro

In [6]:
import spacy

# import the model
import en_core_web_sm

ModuleNotFoundError: No module named 'en_core_web_sm'

In [8]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup

# Launch the browser
browser = Browser('edge')

In [4]:
# Set the URL to visit to a variable
url = 'https://www.yelp.com/dataset/download/'

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Find all divs that contain a question
question_divs = soup.find_all('div', class_="s-post-summary js-post-summary")

In [7]:
question_divs

[]

In [6]:
# Find the summary text of the first question
question_divs[0].find("a", class_="s-link").text

IndexError: list index out of range

In [ ]:
# Close the browser
browser.quit()

In [3]:
yelp_batch = []
var_dtypes = {"stars": np.float16, 
            "useful": np.int32, 
            "funny": np.int32,
            "cool": np.int32,
           }
with open("https://www.yelp.com/dataset/download/yelp_academic_dataset_review.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=var_dtypes, chunksize=1000)
        
    for chunk in reader:
        reduced_chunk = chunk.drop(columns=['review_id', 'user_id'])\
                             .query("`date` >= '2017-12-01'")
        yelp_batch.append(reduced_chunk)
    
yelp_batch = pd.concat(yelp_batch, ignore_index=True)

OSError: [Errno 22] Invalid argument: 'https://www.yelp.com/dataset/download/yelp_academic_dataset_review.json'